### 목록
* 준비
    * [사용할 라이브러리 불러오기](#lib)
    * [엘라스틱서치 환경 설정하기](#es_setting)
    * [엘라스틱서치에서 데이터 조회하기](#es_load)
    * [조회한 데이터를 테이블 형태로 만들기](#df)
* Sibling Pipeline Aggregation
    * [Sum Bucket Aggregation](#sum)
    * [Average Bucket Aggregation](#avg)
    * [Min Bucket Aggregation](#min)
    * [Max Bucket](#max)
---

### 준비

<a name='lib'></a>
#### 사용할 라이브러리 불러오기

In [1]:
import pandas as pd
from elasticsearch import Elasticsearch
import datetime

<a name='es_setting'></a>
#### 엘라스틱서치 환경 설정하기

In [2]:
# 적당한 값을 넣지 않으면 오류가 납니다
es = Elasticsearch()
es_index = 'test_index'
es_type = 'test_type'

<a name='es_load'></a>
#### 엘라스틱서치에서 데이터 조회하기

In [3]:
x = es.search(
      index='test_index', 
      doc_type='test_type', 
      body={
        '_source' : ['주문시간', '상품가격', '판매자평점'],
        'query' : {
            'match_all' : {}
        },
        'size' : 100
      }
)

<a name='df'></a>
#### 조회한 데이터를 테이블 형태로 만들기

In [4]:
df = pd.DataFrame([x['hits']['hits'][idx]['_source'] for idx, _ in enumerate(x['hits']['hits'])])
df

,상품가격,주문시간,판매자평점
0,8000,2017-11-01T05:45:01,1
1,13000,2017-11-09T16:58:07,3
2,22000,2017-11-06T20:03:26,1
3,21000,2017-11-03T18:29:35,3
4,21000,2017-11-09T22:37:36,4
5,13000,2017-11-05T00:41:55,3
6,15000,2017-11-06T00:27:44,1
7,12000,2017-11-09T00:40:16,3
8,10000,2017-11-06T10:28:26,5
9,17000,2017-11-08T02:00:16,3


### Parent Piepline Aggregation

In [5]:
df['날짜'] = pd.to_datetime(df['주문시간'])
df['날짜'] = df['날짜'].apply(lambda x: '월'.join([str(x.month), str(x.day)]) + '일')
df['평점군'] = df['판매자평점'].apply(lambda x: '저평점' if 0<=x<=3 else '고평점')

<a name="sum"></a>
#### Sum Bucket Aggregation
* Bucket별로 Metric Aggreagtion(= bucket 별 대표값 1개 생성 후) 수행 후 Sum한다
* 예시
    * 일 별로
    * 평점 그룹 별로
        * 저평점 : 0~3
        * 고평점 : 4~5
    * 상품가격의 평균을 구하고 (Metric Aggregation : Average)
    * 그 평균(=상품가격들의 평균들의)들의 합을 구하시오

In [6]:
# 일 별로, 평점 그룹 별로 상품 가격의 평균을 구하고...
df_gb_rate = df.groupby(['날짜', '평점군'], as_index=False).mean()[['날짜', '평점군', '상품가격']]
df_gb_rate['상품가격'] = df_gb_rate['상품가격'].apply(lambda x: int(x))
df_gb_rate

,날짜,평점군,상품가격
0,11월10일,고평점,19500
1,11월10일,저평점,18000
2,11월1일,고평점,27000
3,11월1일,저평점,8000
4,11월2일,고평점,10000
5,11월2일,저평점,20000
6,11월3일,고평점,28000
7,11월3일,저평점,17750
8,11월4일,고평점,19333
9,11월4일,저평점,15500


In [7]:
# ... 합을 구하시오
df_gb_rate.groupby(['날짜']).sum().astype(int)

,상품가격
날짜,
11월10일,37500
11월1일,35000
11월2일,30000
11월3일,45750
11월4일,34833
11월5일,42000
11월6일,28200
11월7일,20250
11월8일,40500


<a name="avg"></a>
#### Average Bucket Aggregation
* Bucket별로 Metric Aggreagtion(= bucket 별 대표값 1개 생성 후) 수행 후 Average 한다
* 예시
    * 일 별로
    * 평점 그룹 별로
        * 저평점 : 0~3
        * 고평점 : 4~5
    * 상품가격의 합을 구하고 (Metric Aggregation : Sum)
    * 그 합(=상품가격들의 합들의)들의 평균을 구하시오

In [8]:
# 일 별로, 평점 그룹 별로 상품 가격의 평균을 구하고...
df_gb_rate = df.groupby(['날짜', '평점군'], as_index=False).sum()[['날짜', '평점군', '상품가격']]
df_gb_rate['상품가격'] = df_gb_rate['상품가격'].apply(lambda x: int(x))
df_gb_rate

,날짜,평점군,상품가격
0,11월10일,고평점,39000
1,11월10일,저평점,54000
2,11월1일,고평점,27000
3,11월1일,저평점,8000
4,11월2일,고평점,20000
5,11월2일,저평점,40000
6,11월3일,고평점,28000
7,11월3일,저평점,71000
8,11월4일,고평점,58000
9,11월4일,저평점,62000


In [9]:
# ... 평균을 구하시오
df_gb_rate.groupby(['날짜']).mean().astype(int)

,상품가격
날짜,
11월10일,46500
11월1일,17500
11월2일,30000
11월3일,49500
11월4일,60000
11월5일,40000
11월6일,50500
11월7일,21000
11월8일,40500


<a name="min"></a>
#### Min Bucket Aggregation
* Bucket별로 Metric Aggreagtion(= bucket 별 대표값 1개 생성 후) 수행 후 최소값을 구한다
* 예시
    * 일 별로
    * 평점 그룹 별로
        * 저평점 : 0~3
        * 고평점 : 4~5
    * 상품가격의 평균을 구하고 (Metric Aggregation : Average)
    * 그 평균(=상품가격들의 평균들의)들의 최소값을 구하시오

In [10]:
# 일 별로, 평점 그룹 별로 상품 가격의 평균을 구하고...
df_gb_rate = df.groupby(['날짜', '평점군'], as_index=False).mean()[['날짜', '평점군', '상품가격']]
df_gb_rate['상품가격'] = df_gb_rate['상품가격'].apply(lambda x: int(x))
df_gb_rate

,날짜,평점군,상품가격
0,11월10일,고평점,19500
1,11월10일,저평점,18000
2,11월1일,고평점,27000
3,11월1일,저평점,8000
4,11월2일,고평점,10000
5,11월2일,저평점,20000
6,11월3일,고평점,28000
7,11월3일,저평점,17750
8,11월4일,고평점,19333
9,11월4일,저평점,15500


In [11]:
# ... 최소값을 구하시오
df_gb_rate.groupby(['날짜']).min()[['상품가격']]

,상품가격
날짜,
11월10일,18000
11월1일,8000
11월2일,10000
11월3일,17750
11월4일,15500
11월5일,19000
11월6일,10000
11월7일,7250
11월8일,18500


<a name="max"></a>
#### Max Bucket Aggregation
* Bucket별로 Metric Aggreagtion(= bucket 별 대표값 1개 생성 후) 수행 후 최대값을 구한다
* 예시
    * 일 별로
    * 평점 그룹 별로
        * 저평점 : 0~3
        * 고평점 : 4~5
    * 상품가격의 평균을 구하고 (Metric Aggregation : Average)
    * 그 평균(=상품가격들의 평균들의)들의 최대값을 구하시오

In [12]:
# 일 별로, 평점 그룹 별로 상품 가격의 평균을 구하고...
df_gb_rate = df.groupby(['날짜', '평점군'], as_index=False).mean()[['날짜', '평점군', '상품가격']]
df_gb_rate['상품가격'] = df_gb_rate['상품가격'].apply(lambda x: int(x))
df_gb_rate

,날짜,평점군,상품가격
0,11월10일,고평점,19500
1,11월10일,저평점,18000
2,11월1일,고평점,27000
3,11월1일,저평점,8000
4,11월2일,고평점,10000
5,11월2일,저평점,20000
6,11월3일,고평점,28000
7,11월3일,저평점,17750
8,11월4일,고평점,19333
9,11월4일,저평점,15500


In [13]:
# ... 최대값을 구하시오
df_gb_rate.groupby(['날짜']).max()[['상품가격']]

,상품가격
날짜,
11월10일,19500
11월1일,27000
11월2일,20000
11월3일,28000
11월4일,19333
11월5일,23000
11월6일,18200
11월7일,13000
11월8일,22000
